In [56]:
# 인스타그램 크롤링 코드

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import datetime
from time import sleep
import time
import random

class insta:
    def __init__(self):
        options = webdriver.ChromeOptions()
        options.add_argument('headless')
        options.add_argument('window-size=1920x1080')
        options.add_argument("disable-gpu")
        options.add_argument("lang=ko_KR") 
        options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36") 
        self.driver=webdriver.Chrome("C:/Users/sunkeun_jo/Desktop/chromedriver",options=options)
        
    def login(self,id,pw):
        '''로그인 및 알림설정 제거 함수 '''
        self.driver.find_element_by_xpath("//input[@name='username']").send_keys(id)
        self.driver.find_element_by_xpath("//input[@name='password']").send_keys(pw)
        self.driver.find_element_by_xpath("//button[@type='submit']").submit()
        #알림 제거
#         try:
#             element = WebDriverWait(self.driver, 60).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.mt3GC > button.aOOlW.HoLwm")))

#         finally:       
#             self.driver.find_element_by_css_selector("div.mt3GC > button.aOOlW.HoLwm").click()
        
    def list_to_csv(self,dict_):
        # 가져온 list를 csv파일 형태로 저장
        dt=datetime.datetime.now()
        filename="posts_of_the_airbnb2"+dt.strftime("%Y_%m_%d")
        df=pd.DataFrame(dict_)
        df.to_csv(filename+".csv")
        
    def content_or_not(self,soup):
        try:
            if self.driver.find_element_by_css_selector("div.C7I1f.X7jCj div.C4VMK span"):
                print("yes, there is ")
            return True
        
        except: 
            print("No, there isn't")
            return False
        
    def hashtag_or_not(self,soup):
        try: 
            if self.driver.find_element_by_css_selector("li.gElp9.rUo9f.PpGvg div.C4VMK span > a"):
                print("yes, there is")
            return True 
        except:
            print("No, there isn't")
            return False
    
    def like_or_not(self, soup):
        '''좋아요 확인하는 함수'''
        try:    
            if self.driver.find_element_by_css_selector("div.Nm9Fw span"):
                print("yes, there is")
            return True 
        except:
            print("No, there isn't")
            return False
        
    def view_or_not(self,soup):
        '''view 갯수 확인하는 함수'''
        try:    
            if self.driver.find_element_by_css_selector("div.eo2As span.vcOH2 span"):
                print("yes, there is")
            return True 
        except:
            print("No, there isn't")
            return False
        
    def posting_date(self,soup):
        '''포스팅 날짜가 언제인지 알려주는 함수'''
        try:    
            if self.driver.find_element_by_css_selector("div.k_Q0X.NnvRN time"):
                print("yes, there is")
            return True 
        except:
            print("No, there isn't")
            return False
        
    
    def marketer_generated_content(self,html,i):
        '''marketer가 instagram에 올려놓은 정보를 가지고 오는 함수'''
        content_dict={}
        soup=BeautifulSoup(html,"html.parser")
        
        #post_id
        content_dict["id"]=i
        
        #content writing
        if self.content_or_not(soup): # content가 존자한다면
            marketer_content=[]
            content=soup.select("div.C7I1f.X7jCj div.C4VMK > span")
            for i in content:
                marketer_content.append(i.text)
                content_dict['content']=marketer_content
                
        else : #content가 존재하지 않는다면 
            content_dict['content']=[]
            print("content가 없읍")
        
        # hashtag writing
        if self.hashtag_or_not(soup):
            hash_list=[]
            content=soup.select("div.C7I1f.X7jCj div.C4VMK span > a")
            for i in content:
                hash_list.append(i.text)
                content_dict['hashtag']=hash_list
        else: 
            content_dict['hashtag']=[]
            print("hashtag도 없다")
        
        
        #posting_date
        if self.posting_date(soup):
            data=soup.select("div.k_Q0X.NnvRN time")
            content_dict["date"]=data[0]['datetime']
        else:
            content_dict["date"]="NaN"
            print("포스팅된 날짜가 없네요")
        
        #number of like on the post
        if self.like_or_not(soup):
            data=soup.select("div.Nm9Fw span")
            content_dict['likes']=data[0].text
        else:
            content_dict['likes']=0
            print("like가 없다")
             
        
        #number of view in the case of video post 
        if self.view_or_not(soup):
            data=soup.select("div.eo2As span.vcOH2 span")
            content_dict['views']=data[0].text
        else:
            content_dict['views']=0
        
            
        
        return content_dict
        

    def page_loading(self):
        '''page가 로드하는 것을 기다리는 함수'''
        try:
            element=WebDriverWait(self.driver, 600).until(EC.presence_of_element_located((By.CSS_SELECTOR, "li.gElp9.rUo9f.PpGvg div.C4VMK span")))
        finally:
            print("load is finished")# load 확인
            
    def instacrawl(self,url,_crawl_list):
        crawl_list={}
        self.driver.get(url)
        sleep(3)# sleep을 줌으로써 읽을 수 있게 해줍니다. 무조건 3초 주세요
        self.login("tjsrms1994@naver.com","dhrguswm1!")
        #이제 찾아갑시다. 특정 사이트에 들어가서 값들을 가지고 옵시다. ex) dutch_and_bean
        for i in _crawl_list:
            print("{} 사이트에 들어갑니다.".format(i))
            self.driver.get(i)# 특정사이트에 들어갑니다
            sleep(1.5)
            start_time=time.time()
            #인스타그램 특성상 scroll보다는 하나씩 넘겨가는게 더 좋을 것 같다!
            self.driver.find_element_by_css_selector("div.v1Nh3.kIKUG._bz0w div._9AhH0").click()
            i=1
            post_content_urls=[]
            while True:
                try:
                    print("{}번째 page입니다".format(i))
                    
                    #기다린 다음에 page를 받읍시다
                    self.page_loading()
                
                    html=self.driver.page_source
                    #----ok--- 
                    # 저장된 날짜 이후엔 따로 저장 
                    soup=BeautifulSoup(html,"html.parser")
                    if pd.to_datetime(soup.select("div.k_Q0X.NnvRN time")[0]['datetime'])>=pd.to_datetime("2019-06-13T13:12:58.000Z"):
                        i+=1
                        print(i,"여기인가")
                        sleep(5)
                        self.driver.find_element_by_css_selector("div._2dDPU.CkGkG a._65Bje.coreSpriteRightPaginationArrow").click()
                        continue
                   
                    else:# 저장된 날짜 이후엔 다시 시작합시다
                    
                        print("여기인가")
                        #page를 받은 뒤 회사가 무엇을 썻는지, 어떤 태그를 했는지 봅시다.

                        content=self.marketer_generated_content(html,i)#dictionary 형태로 나온다
                        post_content_urls.append(content)

                    
                        i+=1

                        sleep(5)
    #                     sleep(random.randrange(5,60))
                        # 그 다음으로 넘어가기 
                        self.driver.find_element_by_css_selector("div._2dDPU.CkGkG a._65Bje.coreSpriteRightPaginationArrow").click()
                    
                    
                    
                except : # 만약 에러가 나온다면 
                    print("{}번째 hosting에서 error 발생 뭔가가 잘못됨".format(i))
                    break #나와라 
                    
                    
            
            print(post_content_urls)
            self.list_to_csv(post_content_urls)
            print("이제 끝났습니다 걸린시간은 {}".format(time.time()-start_time))
       

        pass



if __name__=="__main__":
    _lst=['https://www.instagram.com/airbnb']
    crawler=insta()
    crawler.instacrawl("https://www.instagram.com/",_lst)
    

https://www.instagram.com/airbnb 사이트에 들어갑니다.
1번째 page입니다
load is finished
2 여기인가
2번째 page입니다
load is finished
3 여기인가
3번째 page입니다
load is finished
4 여기인가
4번째 page입니다
load is finished
5 여기인가
5번째 page입니다
load is finished
6 여기인가
6번째 page입니다
load is finished
7 여기인가
7번째 page입니다
load is finished
8 여기인가
8번째 page입니다
load is finished
9 여기인가
9번째 page입니다
load is finished
10 여기인가
10번째 page입니다
load is finished
11 여기인가
11번째 page입니다
load is finished
12 여기인가
12번째 page입니다
load is finished
13 여기인가
13번째 page입니다
load is finished
14 여기인가
14번째 page입니다
load is finished
15 여기인가
15번째 page입니다
load is finished
16 여기인가
16번째 page입니다
load is finished
17 여기인가
17번째 page입니다
load is finished
18 여기인가
18번째 page입니다
load is finished
19 여기인가
19번째 page입니다
load is finished
20 여기인가
20번째 page입니다
load is finished
21 여기인가
21번째 page입니다
load is finished
22 여기인가
22번째 page입니다
load is finished
23 여기인가
23번째 page입니다
load is finished
24 여기인가
24번째 page입니다
load is finished
25 여기인가
25번째 page입니다
load is finished
26 여기인가
26번째 page입니다
load is f

load is finished
211 여기인가
211번째 page입니다
load is finished
212 여기인가
212번째 page입니다
load is finished
213 여기인가
213번째 page입니다
load is finished
214 여기인가
214번째 page입니다
load is finished
215 여기인가
215번째 page입니다
load is finished
216 여기인가
216번째 page입니다
load is finished
217 여기인가
217번째 page입니다
load is finished
218 여기인가
218번째 page입니다
load is finished
219 여기인가
219번째 page입니다
load is finished
220 여기인가
220번째 page입니다
load is finished
221 여기인가
221번째 page입니다
load is finished
222 여기인가
222번째 page입니다
load is finished
223 여기인가
223번째 page입니다
load is finished
224 여기인가
224번째 page입니다
load is finished
225 여기인가
225번째 page입니다
load is finished
226 여기인가
226번째 page입니다
load is finished
227 여기인가
227번째 page입니다
load is finished
228 여기인가
228번째 page입니다
load is finished
229 여기인가
229번째 page입니다
load is finished
230 여기인가
230번째 page입니다
load is finished
231 여기인가
231번째 page입니다
load is finished
232 여기인가
232번째 page입니다
load is finished
233 여기인가
233번째 page입니다
load is finished
234 여기인가
234번째 page입니다
load is finished
235 여기인가
235번째 page입니다


In [3]:
import pandas as pd
df=pd.read_csv("posts_of_the_airbnb22020_04_03.csv",index_col=[0])
df

,content,date,hashtag,id,likes,views
0,['Our hosts have been asking how they can help...,2020-03-26T13:50:08.000Z,[],1,"12,362",0
1,['We know you may have questions about the cor...,2020-03-13T22:08:47.000Z,[],2,"23,631",0
2,['$1 million among 10 winners. The Unique Airb...,2020-03-11T16:16:14.000Z,['@jthavn'],3,"27,591",0
3,"['Where there’s a will, there’s a way. We’ve c...",2020-03-10T15:03:53.000Z,[],4,0,"96,927"
4,['Superhost Pamela’s riverfront retreat lets y...,2020-03-09T19:00:11.000Z,['@frankiesaunders'],5,"19,825",0
5,['Textile designer Abril didn’t set out to cre...,2020-03-08T16:10:56.000Z,[],6,"13,783",0
6,['Tucked among Japanese maples in the Little F...,2020-03-07T16:00:12.000Z,[],7,"14,048",0
7,['Superhost Morgan’s villa gives you 180-degre...,2020-03-05T15:21:13.000Z,['@megaann.p'],8,"65,558",0
8,['After an action-packed day on and around Lak...,2020-03-04T16:27:34.000Z,['@terrieleeee'],9,0,"101,924"
9,['Don’t let your private pool keep you from ta...,2020-03-03T16:00:12.000Z,['@haleymahl21'],10,"26,164",0


In [4]:
df['content'].isnull().sum()

0

In [20]:
# 인스타그램 크롤링 코드
# 이전 것 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import datetime
from time import sleep
import time
import random

class insta:
    def __init__(self):
        options = webdriver.ChromeOptions()
        options.add_argument('headless')
        options.add_argument('window-size=1920x1080')
        options.add_argument("disable-gpu")
        options.add_argument("lang=ko_KR") 
        options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36") 
        self.driver=webdriver.Chrome("C:/Users/sunkeun_jo/Desktop/chromedriver",options=options)
        
    def login(self,id,pw):
        '''로그인 및 알림설정 제거 함수 '''
        self.driver.find_element_by_xpath("//input[@name='username']").send_keys(id)
        self.driver.find_element_by_xpath("//input[@name='password']").send_keys(pw)
        self.driver.find_element_by_xpath("//button[@type='submit']").submit()
        #알림 제거
#         try:
#             element = WebDriverWait(self.driver, 60).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.mt3GC > button.aOOlW.HoLwm")))

#         finally:       
#             self.driver.find_element_by_css_selector("div.mt3GC > button.aOOlW.HoLwm").click()
        
    def list_to_csv(self,dict_):
        # 가져온 list를 csv파일 형태로 저장
        dt=datetime.datetime.now()
        filename="posts_of_the_airbnb2"+dt.strftime("%Y_%m_%d")
        df=pd.DataFrame(dict_)
        df.to_csv(filename+".csv")
        
    def content_or_not(self,soup):
        try:
            if self.driver.find_element_by_css_selector("div.C7I1f.X7jCj div.C4VMK span"):
                print("yes, there is ")
            return True
        
        except: 
            print("No, there isn't")
            return False
        
    def hashtag_or_not(self,soup):
        try: 
            if self.driver.find_element_by_css_selector("li.gElp9.rUo9f.PpGvg div.C4VMK span > a"):
                print("yes, there is")
            return True 
        except:
            print("No, there isn't")
            return False
    
    def like_or_not(self, soup):
        '''좋아요 확인하는 함수'''
        try:    
            if self.driver.find_element_by_css_selector("div.Nm9Fw span"):
                print("yes, there is")
            return True 
        except:
            print("No, there isn't")
            return False
        
    def view_or_not(self,soup):
        '''view 갯수 확인하는 함수'''
        try:    
            if self.driver.find_element_by_css_selector("div.eo2As span.vcOH2 span"):
                print("yes, there is")
            return True 
        except:
            print("No, there isn't")
            return False
        
    def posting_date(self,soup):
        '''포스팅 날짜가 언제인지 알려주는 함수'''
        try:    
            if self.driver.find_element_by_css_selector("div.k_Q0X.NnvRN time"):
                print("yes, there is")
            return True 
        except:
            print("No, there isn't")
            return False
        
    
    def marketer_generated_content(self,html,i):
        '''marketer가 instagram에 올려놓은 정보를 가지고 오는 함수'''
        content_dict={}
        soup=BeautifulSoup(html,"html.parser")
        
        #post_id
        content_dict["id"]=i
        
        #content writing
        if self.content_or_not(soup): # content가 존자한다면
            marketer_content=[]
            content=soup.select("div.C7I1f.X7jCj div.C4VMK > span")
            for i in content:
                marketer_content.append(i.text)
                content_dict['content']=marketer_content
                
        else : #content가 존재하지 않는다면 
            content_dict['content']=[]
            print("content가 없읍")
        
        # hashtag writing
        if self.hashtag_or_not(soup):
            hash_list=[]
            content=soup.select("div.C7I1f.X7jCj div.C4VMK span > a")
            for i in content:
                hash_list.append(i.text)
                content_dict['hashtag']=hash_list
        else: 
            content_dict['hashtag']=[]
            print("hashtag도 없다")
        
        
        #posting_date
        if self.posting_date(soup):
            data=soup.select("div.k_Q0X.NnvRN time")
            content_dict["date"]=data[0]['datetime']
        else:
            content_dict["date"]="NaN"
            print("포스팅된 날짜가 없네요")
        
        #number of like on the post
        if self.like_or_not(soup):
            data=soup.select("div.Nm9Fw span")
            content_dict['likes']=data[0].text
        else:
            content_dict['likes']=0
            print("like가 없다")
             
        
        #number of view in the case of video post 
        if self.view_or_not(soup):
            data=soup.select("div.eo2As span.vcOH2 span")
            content_dict['views']=data[0].text
        else:
            content_dict['views']=0
        
            
        
        return content_dict
        

    def page_loading(self):
        '''page가 로드하는 것을 기다리는 함수'''
        try:
            element=WebDriverWait(self.driver, 1200).until(EC.presence_of_element_located((By.CSS_SELECTOR, "li.gElp9.rUo9f.PpGvg div.C4VMK span")))
        finally:
            print("load is finished")# load 확인
            
    def instacrawl(self,url,_crawl_list):
        crawl_list={}
        self.driver.get(url)
        sleep(3)# sleep을 줌으로써 읽을 수 있게 해줍니다. 무조건 3초 주세요
        self.login("tjsrms1994@naver.com","dhrguswm1!")
        #이제 찾아갑시다. 특정 사이트에 들어가서 값들을 가지고 옵시다. ex) dutch_and_bean
        for i in _crawl_list:
            print("{} 사이트에 들어갑니다.".format(i))
            self.driver.get(i)# 특정사이트에 들어갑니다
            sleep(1.5)
            start_time=time.time()
            #인스타그램 특성상 scroll보다는 하나씩 넘겨가는게 더 좋을 것 같다!
            self.driver.find_element_by_css_selector("div.v1Nh3.kIKUG._bz0w div._9AhH0").click()
            i=1
            post_content_urls=[]
            while True:
                try:
                    print("{}번째 page입니다".format(i))
                    
                    #기다린 다음에 page를 받읍시다
                    self.page_loading()
                
                    html=self.driver.page_source
                    #----ok---
                    #page를 받은 뒤 회사가 무엇을 썻는지, 어떤 태그를 했는지 봅시다.
                    
                    content=self.marketer_generated_content(html,i)#dictionary 형태로 나온다
                    post_content_urls.append(content)
                    

                    
                    
                    i+=1
                    
#                     sleep(5)
                    sleep(random.randrange(5,60))
                    # 그 다음으로 넘어가기 
                    self.driver.find_element_by_css_selector("div._2dDPU.CkGkG a._65Bje.coreSpriteRightPaginationArrow").click()
                    
                    
                    
                except : # 만약 에러가 나온다면 
                    print("{}번째 hosting에서 error 발생 뭔가가 잘못됨".format(i))
                    break #나와라 
                    
                    
            
            print(post_content_urls)
            self.list_to_csv(post_content_urls)
            print("이제 끝났습니다 걸린시간은 {}".format(time.time()-start_time))
       

        pass



if __name__=="__main__":
    _lst=['https://www.instagram.com/airbnb']
    crawler=insta()
    crawler.instacrawl("https://www.instagram.com/",_lst)
    

yes, there is
{'as': 'a'}


In [55]:
# print(type(datetime.datetime(df['date'][0])))#"2019-06-13T13:12:58.000Z"
a=pd.to_datetime("2019-06-13T13:12:58.000Z")
a>=pd.to_datetime("2019-06-14T13:11:44.000Z")

False